In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import csv
import os

In [94]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print('Device: ' + device)

Device: cpu


In [95]:
batch_size = 100
n = 12

x_file_path = 'Datasets/kryptonite-' + str(n) + '-X.npy'
y_file_path = 'Datasets/kryptonite-' + str(n) + '-y.npy'

In [96]:
x_raw = torch.tensor(np.load(x_file_path), dtype=torch.float32)
y_raw = torch.tensor(np.load(y_file_path), dtype=torch.float32)

# print(torch.unique(y_raw))

row_count = x_raw.shape[0]
print(f'Row Count: {row_count}')

x_train, x_val = torch.tensor_split(x_raw, [round(row_count * 0.8)], dim=0)
y_train, y_val = torch.tensor_split(y_raw, [round(row_count * 0.8)], dim=0)

train_dataset = torch.utils.data.TensorDataset(x_train.to(device), y_train.to(device))
val_dataset = torch.utils.data.TensorDataset(x_val.to(device), y_val.to(device))

loaders = {
            'train' : torch.utils.data.DataLoader(train_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True, 
                                                num_workers=1),
            
            'validation'  : torch.utils.data.DataLoader(val_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False, 
                                                num_workers=1),
        }


Row Count: 24000


In [97]:
model_struct = [n, 15, 12, 4, 1]

class kryptonite_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(model_struct[0], model_struct[1])
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(model_struct[1], model_struct[2])
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(model_struct[2], model_struct[3])
        self.act3= nn.ReLU()
        self.output = nn.Linear(model_struct[3], model_struct[4])
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.act_output(self.output(x))
        return x
 
model = kryptonite_nn()
# print(model)

In [ ]:
loss_func = 'BCE'           # Just used for recording in csv file
optim_func = 'Adagrad'     # Just used for recording in csv file
lr = 0.07

loss_fn = nn.BCELoss()
optimizer = optim.Adagrad(model.parameters(), lr=lr)

n_epochs = 50
 
for epoch in range(n_epochs):
    loss_arr = []
    for data in loaders['train']:
        x_vals, labels = data
        optimizer.zero_grad()
        label_pred = model(x_vals)
        labels = torch.reshape(labels, (batch_size, 1))
        loss = loss_fn(label_pred, labels)
        loss.backward()
        optimizer.step()
        loss_arr.append(loss.item())

    print(f'Finished epoch {epoch}, epoch loss {np.average(loss_arr)}')

Finished epoch 0, epoch loss 0.6932265010351936
Finished epoch 1, epoch loss 0.6928451371689638
Finished epoch 2, epoch loss 0.6850147042423487
Finished epoch 3, epoch loss 0.6594918090850115
Finished epoch 4, epoch loss 0.603953769740959
Finished epoch 5, epoch loss 0.5494265208641688
Finished epoch 6, epoch loss 0.5054631175783774
Finished epoch 7, epoch loss 0.475312734177957
Finished epoch 8, epoch loss 0.4487015134654939
Finished epoch 9, epoch loss 0.423111061565578
Finished epoch 10, epoch loss 0.40084678772836924
Finished epoch 11, epoch loss 0.3835740591554592
Finished epoch 12, epoch loss 0.37097128418584663
Finished epoch 13, epoch loss 0.36011227313429117
Finished epoch 14, epoch loss 0.3377916032914072
Finished epoch 15, epoch loss 0.3265215326100588
Finished epoch 16, epoch loss 0.31986322975717485
Finished epoch 17, epoch loss 0.31424223026260734
Finished epoch 18, epoch loss 0.31001006431567174
Finished epoch 19, epoch loss 0.30660335773912567
Finished epoch 20, epoch l

In [99]:
train_accuracy = np.array([])
with torch.no_grad():
    for data in loaders['train']:
        x_vals, labels = data
        output = model.forward(x_vals)

        train_accuracy = np.concatenate((train_accuracy, torch.eq(torch.flatten(output.round()), labels).numpy() ))

train_accuracy = train_accuracy.mean()

print(f"Training Accuracy: {train_accuracy}")

Training Accuracy: 0.8965625


In [100]:
y_pred = np.array([])
with torch.no_grad():
    for data in loaders['validation']:
        x_vals, labels = data
        output = model.forward(x_vals)
        # y_pred = np.concatenate((y_pred, output.numpy().flatten()))
        y_pred = np.concatenate((y_pred, torch.eq(torch.flatten(output.round()), labels).numpy() ))

val_accuracy = y_pred.mean()        
# val_accuracy = (y_pred.round() == y_val).float().mean()
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.89125


In [101]:
# Save model params to csv file

csv_file_path = 'nn_model_record.csv'

fields = ['n', 'train_acc', 'val_acc', 'optim_func', 'learning_rate', 'loss_func', 'epochs', 'batch_size', 'model_struct']

if not os.path.isfile(csv_file_path):
    with open(csv_file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)

with open(csv_file_path, mode='a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([n, round(train_accuracy.item(), 3), round(val_accuracy.item(), 3), optim_func, lr, loss_func, n_epochs, batch_size, model_struct])

    